In [4]:
# Skriptas sukuria zive duomenų analogą iš MIT duomenų, į sub-aplanką rec_dir įrašomi
# transformuoti EKG įrašai *.npy ir anotacijų failai *.json.
# # Išeities duomenys MIT_BIH atsisiunčiami iš aplanko mit-bih-arrhythmia-database-1.0.0
# Anotacijų failuose paliktos iš MIT tik anotacijos N,L,R,e,j,V,E,S,A,a,J,F,
# visas kitas perpavadiname į 'Q' (Unclassifiable beat).
# Grupuojant paliktas anotacijas į Zive atitikmenis, paliekame N,S,V,U anotacijas.
# F anotacijas, kurios yra: "Fusion of ventricular and normal beat (Fusion PVC) -F"
# priskiriame U. 

# Pagrindinis sukūriamas aplankas - db_path = pvz. 'mit2zive-arrhythmia-database-1.0.0'.

# Programos planas:
# Nuskaitome MIT-BIH ekg įrašą
# Pakeičiame diskretizavimo dažnį iš 360 į 200 Hz
# Perskaičiuojame anotacijos vietas
# Anotacijų mapingas į N,V,S,U: agreguojame anotacijas, kad jos atitiktų Zive anotacijas.
# Pastaba: F anotacijos, kurias neaišku, kur priskirti - priskiriamos prie prie U. 
# Įrašome ekg ir anotacijas zive formatu į diską


# //////////////////////////////////////////////////////////////////////////////////////////////////////
#  
import pandas as pd
import numpy as np
import wfdb

import matplotlib.pyplot as plt
import json
import sys
from icecream import ic
from pathlib import Path
from neurokit2 import signal_resample
from collections import Counter
from collections import Counter

from zive_util_vu import create_dir
from zive_util_vu import read_rec_attrib

# from mit2zive_util import ecg_record_plot
# from mit2zive_util import plot_sign

# Maksimumo vietos lyginimui 
def max_place(SubjCode, atr_sample, sign, sample_from, sample_to):  
    for i in range(len(atr_sample)):
        index_s = atr_sample[i]
        if (index_s < sample_from or index_s >sample_to):
            continue
        print(f"\n{i}, index_s: {index_s}  max: {sign[index_s]:.3f}")
        for j in range(-5,5):
            index_y = index_s+j
            formatted_float = "{:.3f}".format(sign[index_y])
            print(index_y, formatted_float, end =' ')

def print_distribut_of_beat_annot_senas(rec_dir, records_nr, beats_annot):

    # Pasiruošimas ciklui per pacientų įrašus
    # print(list(beats_annot.keys()))
    labels_rec_all = pd.DataFrame(columns = list(beats_annot.keys()), dtype=int)
    # print(labels_rec_all.shape)
    labels_rec_all.insert(0, 'SubjCode', 0)

    # Ciklas per pacientų įrašus

    for SubjCode in records_nr:
        # print(SubjCode)
        # Paciento anotacijų nuskaitymas, nustatomas anotacijų 
        # ir užduotų klasių pasiskirstymas per visus įrašus

        # labels_rec = np.zeros(labels_rec_all.shape[1], dtype=int)
        # labels_rec[0] = SubjCode

        # Nuskaitome anotacijas ir jų indeksus
        file_path = Path(rec_dir, str(SubjCode) + '.json')
        df = pd.read_json(file_path, orient = 'records')
        atr_symbol = df['annotationValue'].to_numpy()

        new_dict = dict(zip(Counter(atr_symbol).keys(), Counter(atr_symbol).values()))
        lst_keys = list(beats_annot.keys())
        lst_keys.insert(0, 'SubjCode')
        empty_dict = dict.fromkeys(lst_keys, 0)
        for key in new_dict:
            empty_dict[key] = new_dict[key]
        empty_dict['SubjCode'] = SubjCode    
        tmp = pd.DataFrame([empty_dict])
        labels_rec_all = pd.concat([labels_rec_all, tmp], axis = 0)

    # Ciklo per pacientų įrašus pabaiga

    # Parodomas anotacijų ir užduotų klasių pasiskirstymas per visus įrašus
    # print("labels_rec_all:\n")
    print(labels_rec_all)
    suma = labels_rec_all.iloc[:,1:].sum()
    print('\n',' '*5, 'sum: ',str(suma.tolist())[1:-1])
    total = suma.sum()
    print(' '*5, 'total: ', total)


def print_distribut_of_beat_annot(rec_dir, records_nr, beats_annot):

    # Pasiruošimas ciklui per pacientų įrašus
    # print(list(beats_annot.keys()))
    labels_rec_all = pd.DataFrame(columns = list(beats_annot.keys()), dtype=int)
    # print(labels_rec_all.shape)
    labels_rec_all.insert(0, 'SubjCode', 0)

    # Ciklas per pacientų įrašus

    for SubjCode in records_nr:
        # print(SubjCode)
        # Paciento anotacijų nuskaitymas, nustatomas anotacijų 
        # ir užduotų klasių pasiskirstymas per visus įrašus

        # labels_rec = np.zeros(labels_rec_all.shape[1], dtype=int)
        # labels_rec[0] = SubjCode

        # Nuskaitome anotacijas ir jų indeksus
        atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

        new_dict = dict(zip(Counter(atr_symbol).keys(), Counter(atr_symbol).values()))
        lst_keys = list(beats_annot.keys())
        lst_keys.insert(0, 'SubjCode')
        empty_dict = dict.fromkeys(lst_keys, 0)
        for key in new_dict:
            empty_dict[key] = new_dict[key]
        empty_dict['SubjCode'] = SubjCode    
        tmp = pd.DataFrame([empty_dict])
        labels_rec_all = pd.concat([labels_rec_all, tmp], axis = 0)

    # Ciklo per pacientų įrašus pabaiga

    # Parodomas anotacijų ir užduotų klasių pasiskirstymas per visus įrašus
    # print("labels_rec_all:\n")
    print(labels_rec_all)
    suma = labels_rec_all.iloc[:,1:].sum()
    print('\n',' '*5, 'sum: ',str(suma.tolist())[1:-1])
    total = suma.sum()
    print(' '*5, 'total: ', total)



In [5]:
print("\nSukuriamas Zive duomenų analogas iš MIT duomenų")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI\\DUOM_2022_RUDUO_2'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/DUOM_2022_RUDUO_2'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Aplankas su MIT-BIH duomenų rinkiniu
db_folder_mit = 'mit-bih-arrhythmia-database-1.0.0'

#  Aplankas, kur rašome MIT2ZIVE duomenis
db_folder_mit2zive = 'records_npy'

# Užduodamas pacientų įrašų sąrašas
# records_nr = np.array([100,101])
records_nr = np.array([100, 101, 103, 105, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124,
200, 201, 202, 203, 205, 208, 209, 210, 212, 213, 214, 215, 219, 220, 221, 222, 223, 228, 230, 231, 232, 233, 234])

# MIT_BIH duomenų diskretizavimo dažnumas
fs_mit = 360

# Originalių anotacijų sąrašas
annot_list_orig = ['N','L','R','e','j','A','a','J','S','V','E','F','Q']

# Žodynas grupavimui
grupav_zodynas = {'N':'N', 'V':'V', 'S':'S', 'F':'U', 'L': 'N', 'R': 'N','e':'N', 'j': 'N', 'A': 'S', 'a': 'S', 'J': 'S', 'E':'V', 'Q':'U'}  

# Nuoroda į MIT-BIH duomenų rinkinį
db_path_mit = Path(Duomenu_aplankas, db_folder_mit)

print("Bendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("MIT duomenų aplankas: ", db_folder_mit)
print("Pacientų įrašų sąrašas:\n",records_nr)
print("Paliekamų grupavimui anotacijų sąrašas:\n",annot_list_orig)
print("Žodynas grupavimui:\n", grupav_zodynas)
print("Aplankas, kur rašome MIT2ZIVE duomenis:\n", db_folder_mit2zive)

print("Pastaba: neįtraukti įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers")
print("207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF")
print("\nDiskretizavimo dažnis fs_mit: ", fs_mit)


# MIT2ZIVE duomenų diskretizavimo dažnumas
fs_zive = 200 # diskretizavimo dažnumas

# Nuoroda į aplanką MIT2ZIVE EKG įrašams (.npy) ir anotacijoms (.json)
rec_dir = Path(Duomenu_aplankas, db_folder_mit2zive)

sant = fs_zive/fs_mit

print("\nAplankas transformuotiems duomenims: ", db_folder_mit2zive)

# Sukūriame aplanką transformuotiems iš MIT į Zive formatą duomenims
create_dir(rec_dir)

print("Diskretizavimo dažnis fs_zive: ", fs_zive)
print(f"fs_zive/fs_mit = {sant:.2f}")

# print("Originalios anotacijos įrašuose:", annot_list)

# Vaizdavimo intervalas (sekundėmis)
Interv = 1

# Vaizduojama reikšmių
Reiksm_MIT_360 = Interv*fs_mit 
Reiksm_Zive_200 = Interv*fs_zive

# Kiek reikšmių nupjauname įrašų pradžioje
Tranc_MIT = 0
  
# Kiek reikšmių nupjauname resamplintame įraše
Tranc_Resampl = int(Tranc_MIT*sant)

print(f'\nTranc_MIT, Tranc_Resampl = {Tranc_MIT, Tranc_Resampl} ')


Sukuriamas Zive duomenų analogas iš MIT duomenų
OS in my system :  win32
Bendras Zive duomenų aplankas:  D:\DI\DUOM_2022_RUDUO_2
MIT duomenų aplankas:  mit-bih-arrhythmia-database-1.0.0
Pacientų įrašų sąrašas:
 [100 101 103 105 106 108 109 111 112 113 114 115 116 117 118 119 121 122
 123 124 200 201 202 203 205 208 209 210 212 213 214 215 219 220 221 222
 223 228 230 231 232 233 234]
Paliekamų grupavimui anotacijų sąrašas:
 ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', 'Q']
Žodynas grupavimui:
 {'N': 'N', 'V': 'V', 'S': 'S', 'F': 'U', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N', 'A': 'S', 'a': 'S', 'J': 'S', 'E': 'V', 'Q': 'U'}
Aplankas, kur rašome MIT2ZIVE duomenis:
 records_npy
Pastaba: neįtraukti įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers
207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF

Diskretizavimo dažnis fs_mit:  360

Aplankas transformuotiems duomenims:  records_npy
Directory 'D:\DI\DUOM_2022_RUDUO_2\records_n

In [6]:
# I-a dalis. MIT-BIH duomenų transformacija į MIT2ZIVE npy ir json formatą

# Surandame visas anotacijas, kurios naudojamos MIT duomenyse
print('Surandame visas anotacijas, kurios naudojamos MIT duomenyse\n')

# Ciklas per pacientų įrašus
for SubjCode in records_nr:

    # Nuskaitome įrašą
    subject_path = f'{db_path_mit}/{SubjCode}'
    # ic(subject_path)

    # Įrašo skaitymo variantai
    # sequence, fields = wfdb.rdsamp(subject_path, channels=[0])
    # record = wfdb.rdrecord('sample-data/a103l')

# https://www.programmersought.com/article/28613723297/
    
     # Variantas 1 - skaitmeninės reikšmės
    # record = wfdb.rdrecord(subject_path, sampfrom=1,
    #                             channels=[0], physical=False)
    # sign_raw = record.d_signal[:,0]

    # Variantas 2 - fizinės reikšmės - perverstos į mV
    record = wfdb.rdrecord(subject_path, sampfrom=0,channels=[0], physical=True)
    sign_raw = record.p_signal[:,0]

    # https://wfdb.readthedocs.io/en/latest/wfdb.html

    len_sign = sign_raw.shape[0]
    print("\n")
    print(f"Pacientas iš MIT: {SubjCode}  Reikšmių: {len_sign}")

    # sign_transf = sign_raw
    # sign_transf = signal_filter(signal=sign_raw, sampling_rate=360,
    #  lowcut=0.1, method="butterworth", order=5)


# Nuskaitome originalaus įrašo anotacijas
    # https://wfdb.readthedocs.io/en/latest/wfdb.html 

    ann = wfdb.rdann(subject_path, 'atr', sampfrom=0, sampto=None, shift_samps=False)
    atr_sample = ann.sample
    atr_symbol = np.array(ann.symbol)

# Originalaus įrašo vizualizacija
# Vaizduodami nupjauname įrašo pradžios gabaliuką, užduodame vizualizacijos diapazoną
    sampfrom = Tranc_MIT
    sampto = Reiksm_MIT_360 + Tranc_MIT
    # plot_sign(sign_raw, sampfrom, sampto, ann_samp=atr_sample, ann_symb=atr_symbol,
    #  title= 'Pacientas ' + str(SubjCode) +' Orig 360 Hz', figsize=(18,3))

# //////////////////////////////////// laikinas
# Maksimumo vietos lyginimui
    # print("\nsubject original:", SubjCode)
    # max_place(SubjCode, atr_sample, sign_raw, sampfrom, sampto)   
# //////////////////////////////////// laikinas

# Resampling iš 360 Hz (MIT) į 200 Hz (Zive)
    # if (SubjCode == 100):
    #     print(sign_raw[:20])
    sign_transf = signal_resample(sign_raw, sampling_rate=fs_mit,
                              desired_sampling_rate=200, method="numpy")
    # if (SubjCode == 100):
    #     print(sign_transf[:20])
    # https://neurokit2.readthedocs.io/en/latest/_modules/neurokit2/signal/signal_resample.html

    # print(sign_transf.shape)
    print(f"Pacientas Zive rinkiniui: {SubjCode}  Reikšmių: {sign_transf.shape[0]}")

# Resampliname anotacijų vietas
    atr_sample = (atr_sample*sant).astype(int)

# Nustatome, kokių ir kiek simbolių pasitaiko originaliose anotacijose  
    # print(Counter(ann.symbol).keys()) # equals to list(set(words))
    # print(Counter(ann.symbol).values()) # counts the elements' frequency
    dict_symbols = dict(zip(Counter(ann.symbol).keys(), Counter(ann.symbol).values()))
    print("Anotacijos:", dict_symbols)

# # Paliekame nepakeistas tik užduotas anotacijas iš annot_list, visas kitas perpavadiname į 'Q'
    for i in range(atr_sample.shape[0]):
        if atr_symbol[i] not in annot_list_orig:
            atr_symbol[i] = 'Q'
  
# Resamplinto įrašo vizualizacija
    sampfrom = Tranc_Resampl
    sampto = Reiksm_Zive_200 + Tranc_Resampl
    # plot_sign(sign_transf, sampfrom, sampto, ann_samp=atr_sample, ann_symb=atr_symbol,
    #  title='Pacientas ' + str(SubjCode) +' MIT Resampled 200 Hz', figsize=(18,3))

# //////////////////////////////////// laikinas
# Maksimumo vietos lyginimui
    # print("\nsubject resampled:", SubjCode)
    # max_place(SubjCode, atr_sample, sign_transf, sampfrom, sampto)   
# //////////////////////////////////// laikinas

# Anotacijos perrašomos dict formatu:
    atr_list = [{"sampleIndex":int(atr_sample[i]),"annotationValue":atr_symbol[i]} for i in range(atr_sample.shape[0])]
    # print(atr_list[:20])

# Anotacijos įrašomos į diską
    file_path = Path(rec_dir, str(SubjCode) + '.json')
    with open(file_path, 'w') as f:
        json.dump(atr_list, f)

# Įrašoma EKG
    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    with open(file_path, 'wb') as f:
        np.save(f, sign_transf)

# SKAITYMO TESTAS

# Kontrolinis anotacijų skaitymas
    file_path = Path(rec_dir, str(SubjCode) + '.json')
    df = pd.read_json(file_path, orient = 'records')
    atr_sample = df['sampleIndex'].to_numpy()
    atr_symbol = df['annotationValue'].to_numpy()
    # ic(atr_sample[:20], atr_symbol[:20])

    # print('sign_transf', sign_transf.shape)
    # print(sign_transf[:10])

# Kontrolinis EKG skaitymas
    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    with open(file_path, "rb") as f:
        sign_loaded = np.load(f) 
    # print(sign_loaded[:10])
    # print('sign_loaded', sign_loaded.shape)

# Originalaus įrašo vizualizacija naudojant wfdb analogą
    # plot_sign(sign_loaded, sampfrom, sampto, ann_samp=atr_sample, ann_symb=atr_symbol, 
    # title=str(SubjCode) +' test', figsize=(18,3))

# Nustatomas originalių anotacijų pasiskirstymas per visus įrašus
nr_sequence = list(range(len(annot_list_orig)))
beats_annot = dict(zip(annot_list_orig, nr_sequence))
print("\nPaliktų anotacijų pasiskirstymas per visus įrašus\n")
print_distribut_of_beat_annot_senas(rec_dir, records_nr, beats_annot)

# II-a dalis. Anotacijas stambiname į makro anotacijas, panaudodami žodyną grupav_zodynas

for SubjCode in records_nr:
    file_path = Path(rec_dir, str(SubjCode) + '.json')
    df = pd.read_json(file_path, orient = 'records')
    atr_sample = df['sampleIndex'].to_numpy()
    atr_symbol_orig = df['annotationValue'].to_numpy()    
    atr_symbol = [grupav_zodynas[symb] for symb in atr_symbol_orig]

# Anotacijos perrašomos json formatu:
    atr_list = [{"sampleIndex":int(atr_sample[i]),"annotationValue":atr_symbol[i]} for i in range(atr_sample.shape[0])]
    # print(atr_list[:20])

# Suformuojame dict įrašymui json formatu
    atr_dict = {"noises":[], "rpeaks":atr_list}

# Makro anotacijos įrašomos į diską
    file_path = Path(rec_dir, str(SubjCode) + '.json')
    with open(file_path, 'w') as f:
        json.dump(atr_dict, f)
   
# Nustatomas makro anotacijų pasiskirstymas per visus įrašus.
# Makro anotacijų sąrašas
annot_list_makro = ['N','S','V','U']
nr_sequence = list(range(len(annot_list_makro)))
beats_annot = dict(zip(annot_list_makro, nr_sequence))
print("\nAgreguotų anotacijų pasiskirstymas per visus įrašus\n")
print_distribut_of_beat_annot(rec_dir, records_nr, beats_annot)

# Surandame įrašų, kuriuose yra nurodytos anotacijos, skaičius
all_lst = []
for SubjCode in records_nr:
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)
    smb_lst = list(Counter(atr_symbol))
    # print (index, SubjCode, smb_lst)
    for i in range(len(smb_lst)):
        all_lst.append(smb_lst[i])
# print(all_lst)
smb_lst = Counter(all_lst)
print("\nĮrašų, kuriuose yra nurodytos anotacijos, skaičius:")
print(smb_lst)

# III dalis. Suformuojame info_create masyvą ir įrašome į diską

# Transformuojame pacientų sąrašą įrašymui į info_creat
SubjCodes = records_nr.tolist()
# SubjCodes = [str(x) for x in SubjCodes]

info = {
    'db_folder':db_folder_mit2zive,
    'fs': fs_zive,
    'SubjCodes':SubjCodes,
    'annot_list':annot_list_makro
    }

file_name = Path(rec_dir,'info_create_m.json')
with open(file_name, 'w') as f:
    json.dump(info, f)
print("\nSchemos parametrai įrašyti į failą: ", file_name, "\n" )

all_subjcode_lst = info['SubjCodes'] # pacientų įrašų sąrašas
file_path = Path(rec_dir,'all_subjcode_lst_m.csv')
np.savetxt(file_path, np.array(all_subjcode_lst), delimiter=',', fmt='%d')
print("\nPacientų įrašų kodai įrašyti į failą: ", file_path )

print("\nPabaiga.............")


Surandame visas anotacijas, kurios naudojamos MIT duomenyse



Pacientas iš MIT: 100  Reikšmių: 650000
Pacientas Zive rinkiniui: 100  Reikšmių: 361111
Anotacijos: {'+': 1, 'N': 2239, 'A': 33, 'V': 1}


Pacientas iš MIT: 101  Reikšmių: 650000
Pacientas Zive rinkiniui: 101  Reikšmių: 361111
Anotacijos: {'+': 1, 'N': 1860, '~': 4, '|': 4, 'Q': 2, 'A': 3}


Pacientas iš MIT: 103  Reikšmių: 650000
Pacientas Zive rinkiniui: 103  Reikšmių: 361111
Anotacijos: {'+': 1, 'N': 2082, '~': 6, 'A': 2}


Pacientas iš MIT: 105  Reikšmių: 650000
Pacientas Zive rinkiniui: 105  Reikšmių: 361111
Anotacijos: {'+': 1, 'N': 2526, 'V': 41, '~': 88, '|': 30, 'Q': 5}


Pacientas iš MIT: 106  Reikšmių: 650000
Pacientas Zive rinkiniui: 106  Reikšmių: 361111
Anotacijos: {'~': 30, '+': 41, 'N': 1507, 'V': 520}


Pacientas iš MIT: 108  Reikšmių: 650000
Pacientas Zive rinkiniui: 108  Reikšmių: 361111
Anotacijos: {'+': 1, 'N': 1739, 'V': 17, 'x': 11, '~': 41, '|': 8, 'F': 2, 'A': 4, 'j': 1}


Pacientas iš MIT: 109  Rei